在环境中执行策略 $\pi_{\theta_k}$ 并保存轨迹集 $\mathcal{D}_k=\{\tau_i\}$ 。
```python
# 选择动作和动作对应的对数概率
a, logprob_a = agent.select_action(s, deterministic=False)  # 在训练时使用随机动作
s_next, r, dw, tr, info = env.step(a)  # 执行动作并获取下一个状态、奖励以及其他信息
done = (dw or tr)  # 如果游戏结束（死亡或胜利），则done为True

# 存储当前的转移数据
agent.put_data(s, a, r, s_next, logprob_a, done, dw, idx=traj_length)
s = s_next
```

计算将得到的奖励$\hat{G}_t$。

```python
# 计算当前状态的值函数 vs 和下一个状态的值函数 vs_
vs = self.critic(s)
vs_ = self.critic(s_next)

# 计算时序差分误差（deltas），考虑奖励、折扣因子、下一个状态值函数、权重和完成标志
deltas = r + self.gamma * vs_ * (~dw) - vs
# 将时序差分误差转换为NumPy数组
deltas = deltas.cpu().flatten().numpy()
```

基于当前的价值函数$V_{\phi_k}$计算优势函数$\hat{A}_t$。
```python
# 初始化优势列表，初始值为0
adv = [0]

# 通过反向迭代计算优势（advantage）
for dlt, done in zip(deltas[::-1], done.cpu().flatten().numpy()[::-1]):
    # 计算当前时间步的优势，考虑时序差0.002分误差、折扣因子、权重和完成标志
    advantage = dlt + self.gamma * self.lambd * adv[-1] * (~done)
    # 将计算得到的优势添加到优势列表
    adv.append(advantage)
# 反转优势列表，以保持正确的时间顺序
adv.reverse()

# 剔除优势列表的最后一个元素，以去除冗余添加的初始值
adv = copy.deepcopy(adv[0:-1])
# 将优势列表转换为PyTorch张量，并进行形状调整
adv = torch.tensor(adv).unsqueeze(1).float().to(self.dvc)

# 计算时序差分目标（td_target）
td_target = adv + vs

# 如果启用了优势归一化，对优势进行归一化处理
if self.adv_normalization:
# 计算优势的均值和标准差，并进行归一化处理
    adv = (adv - adv.mean()) / ((adv.std() + 1e-4))
```

for $m\in\{1,\cdots,M\}$ do

$$
\ell_t(\theta^{\prime})=\frac{\pi_\theta(A_t|S_t)}{\pi_{\theta_{\mathrm{old}}}(A_t|S_t)}
$$

```python
# 计算优化迭代次数，确保能够处理所有样本
optim_iter_num = int(math.ceil(s.shape[0] / self.batch_size))

# 迭代执行多个优化步骤（K_epochs次）
for _ in range(self.K_epochs):
    # 随机打乱样本索引的顺序
    perm = np.arange(s.shape[0])
    np.random.shuffle(perm)
    # 将打乱后的索引转换为PyTorch张量并发送到指定设备
    perm = torch.LongTensor(perm).to(self.dvc)

    # 根据打乱后的索引重新排列数据，使用深度复制以避免原始数据被修改
    s, a, td_target, adv, old_prob_a = \
        s[perm].clone(), a[perm].clone(), td_target[perm].clone(), adv[perm].clone(),
    old_prob_a[perm].clone()

    # 遍历每个优化迭代
    for i in range(optim_iter_num):
        # 确定当前迭代的样本索引范围
        index = slice(i * self.batch_size, min((i + 1) * self.batch_size, s.shape[0]))

        # 计算策略的概率分布及其熵
        prob = self.actor.pi(s[index], softmax_dim=1)
        entropy = Categorical(prob).entropy().sum(0, keepdim=True)

        # 计算新旧概率比率、Clipped Surrogate函数及Actor的损失
        # 通过索引操作，获取当前动作对应的概率
        prob_a = prob.gather(1, a[index])
        # 计算新概率与旧概率的比率
        ratio = torch.exp(torch.log(prob_a) - torch.log(old_prob_a[index]))
```

采用 Adam 随机梯度上升算法最大化 PPO-Clip 的目标函数来更新策略：

$$\begin{aligned}\theta_{k+1}&=\arg\max_{\theta}\frac{1}{|\mathcal{D}_k|T}\sum_{\tau\in D_k}\sum_{t=0}^{\mathrm{T}}\min(\ell_t(\theta')A^{\pi\theta_{\mathrm{odd}}}(S_t,A_t),\text{clip}(\ell_t(\theta'),1-\epsilon,1+\epsilon)A^{\pi_{\theta_{\mathrm{old}}}}(S_t,A_t))\end{aligned}$$

```python
# 计算Clipped Surrogate函数的第一部分
surr1 = ratio * adv[index]
# 计算Clipped Surrogate函数的第二部分，使用torch.clamp函数将比率限制在[1 - clip_rate, 1 + clip_rate]范围内
surr2 = torch.clamp(ratio, 1 - self.clip_rate, 1 + self.clip_rate) * adv[index]
# 计算Actor的损失，包括负的Clipped Surrogate函数最小值和熵正则项
a_loss = -torch.min(surr1, surr2) - self.entropy_coef * entropy

# 对Actor进行梯度清零，计算梯度，进行梯度裁剪，更新参数
# 将Actor的梯度置零，以防止梯度累积
self.actor_optimizer.zero_grad()
# 计算Actor的损失对参数的梯度
a_loss.mean().backward()
# 使用梯度裁剪，防止梯度爆炸
torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 40)
# 更新Actor的参数，执行一步优化器
self.actor_optimizer.step()
```

采用梯度下降算法最小化均方误差来学习价值函数：
$$
\phi_{k+1}=\arg\min_{\phi}\frac{1}{|\mathcal{D}_{k}|T}\sum_{\tau\in\mathcal{D}_{k}}\sum_{t=0}^{\mathrm{T}}\left(V_{\phi}(S_{t})-\hat{G}_{t}\right)^{2}
$$

```python
# 计算Critic的损失，包括均方误差损失和L2正则项
# 计算均方误差损失，即Critic网络输出值与目标值的差的平方的均值
c_loss = (self.critic(s[index]) - td_target[index]).pow(2).mean()
# 添加L2正则项，对Critic的权重参数进行平方求和，并乘以L2正则化系数self.l2_reg
for name, param in self.critic.named_parameters():
    if 'weight' in name:
        c_loss += param.pow(2).sum() * self.l2_reg

# 对Critic进行梯度清零，计算梯度，更新参数
self.critic_optimizer.zero_grad()
c_loss.backward()
self.critic_optimizer.step()
```